The transription is done using google speech recognition API. The transcription was not perfect. Some of the repeated errors during transcription is corrected using seperate functions. The quantity of each food item is found using dependency parser. Some of the correct transcription also gave wrong output.  

In [2]:
import os
import spacy
import speech_recognition as sr
from pydub import AudioSegment
import pandas as pd
from word2number import w2n
import re

# Download the spaCy model
spacy.cli.download("en_core_web_sm")

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

C:\Users\dilsh\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:

# Function to convert audio to text
def audio_to_text(file_path):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(file_path)
    audio.export("temp.wav", format="wav")
    
    with sr.AudioFile("temp.wav") as source:
        recognizer.adjust_for_ambient_noise(source)
        audio_data = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Could not understand the audio"
        except sr.RequestError as e:
            return f"Could not request results from Google Web Speech API; {e}"

#  Caterer food item dataset
food_items = ["pizza", "burger", "pasta", "salad", "soda", "sandwich","sadhya", "payasam"]

# Function to extract food items and quantities from text using spaCy
def extract_food_items(text, food_items):
    #To correct some common transcription error occured
    text="i want two number of pizzas and and burgers five in number and 1 salad"
    text = correct_transcription_items(text) 
    text = correct_transcription_quantity(text)
    print('corrected text',text)
    doc = nlp(text.lower()) #several nlp process are performed. a pipeline of events including tokenisation, lemmatization .....
    order = {}
    
    for token in doc:
        if token.lemma_ in food_items:
            print('token lemma',token.lemma_)
            # Find the quantity by checking the token's left children for numbers
            quantity = 1  # Default quantity
            print(f"Token: {token.text}")
            print("Left children:", [child.text for child in token.lefts])
            print("Right children:", [child.text for child in token.rights])
            print()
            for child in token.lefts:
                if child.like_num:
                    quantity = int(w2n.word_to_num(child.text))  #get the numerical value and store to the quantity
                    break
            for child in token.rights:
                if child.like_num:
                    quantity = int(w2n.word_to_num(child.text))
                    break
            if token.lemma_ in order:    #if the perticular token is already in the dictionary the quantity is updated
                order[token.lemma_] += quantity
            else:
                order[token.lemma_] = quantity
    
    return order

# Function to structure extracted data into a DataFrame
def structure_data(order, customer_id):
    df = pd.DataFrame(list(order.items()), columns=["Food Item", "Quantity"])
    df["Customer ID"] = customer_id
    return df
#function to correct some common transciption error of items
def correct_transcription_items(text):
    corrections = {
        r'\bbrothers\b': 'burgers',
        r'\bsadiya\b': 'sadhya',
        r'\bsathya\b': 'sadhya',
        
        
        # Add more specific corrections as needed
    }
    for pattern, replacement in corrections.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text
    
#function to correct some common transcription error of the quantity

def correct_transcription_quantity(text):
    corrections = {
        r'\bto burgers\b': 'two burgers',
        r'\bto pizza\b': 'two pizza',
        r'\bto sadhya\b': 'two sadhya',
        r'\bto payasam\b': 'two payasam',
        r'\bon pizzas\b': 'one pizza',
        r'\bon burger\b': 'one burger',
        r'\bon sadhya\b': 'one sadhya',
        r'\bon payasam\b': 'one payasam',
        # Add more specific corrections as needed
    }
    
    for pattern, replacement in corrections.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text
    
# Function to process the order
def process_order(file_path, customer_id):
    #  Convert audio to text using google speach recognition api 
    text = audio_to_text(file_path)
    print('audio text is', text)
    if "Could not" in text:
        return None

    
    #  Extract food items and quantities
    order = extract_food_items(text, food_items)
    print('order is ',order)
    
    # Step 3: Structure data
    if order:
        df = structure_data(order, customer_id)
        output_file = f"order_{customer_id}.csv"
        df.to_csv(output_file, index=False)
        print(f"Order processed and saved to {output_file}")
    else:
        print("No valid food items found in the order")

# audio to csv file of a recording 
process_order("Recording9.wav", "customer_001")


audio text is I need to Pizza three burgers and one Sathya
corrected text i want two number of pizzas and and burgers five in number and 1 salad
token lemma pizza
Token: pizzas
Left children: []
Right children: []

token lemma burger
Token: burgers
Left children: []
Right children: ['five']

token lemma salad
Token: salad
Left children: ['number']
Right children: []

order is  {'pizza': 1, 'burger': 5, 'salad': 1}
Order processed and saved to order_customer_001.csv
